## 1. 사용할 패키지 불러오기

In [2]:
import pandas as pd
import cv2
import os
from sklearn.model_selection import train_test_split
from torch.optim import Adam
from torch.nn import L1Loss
from data_gen.data_gen import DatasetGenerator
import torch
import torchvision.models as models
import torch.nn as nn
import numpy as np
from tqdm import tqdm

## 2. 데이터 불러오기

### (1) Table 데이터

In [3]:
table_data = pd.read_excel('data/final_data.xlsx')
table_data.head()

,작가생존여부_사망,작가생존여부_생존,작가생존여부_알수없음,판매계절_가을,판매계절_겨울,판매계절_봄,판매계절_여름,재료_견본채색,재료_기타,재료_브론즈,...,판매처_칸옥션,판매처_케이옥션,판매처_헤럴드아트데이,가로,세로,작품 판매 횟수,판매가격,작가명,제목,이미지 고유 번호
0,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0.065372,0.065372,0.0,600000,임상진 Lim SangChin (1935~2013),무제,380410
1,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0.026429,0.026429,0.0,400000,정술원 Jung SulWon (1885~1959),화조,380460
2,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0.018878,0.018878,0.0,100000,정주상 Jeong JuSang (1925~2012),심정흥장 (선면),380491
3,0,1,0,1,0,0,0,0,0,0,...,0,0,0,0.025351,0.025351,0.0,360000,이양원 Lee YangWon (1944~),풍속도,380417
4,0,1,0,1,0,0,0,0,1,0,...,0,0,0,0.019957,0.019957,0.0,240000,이외수 Lee OiSoo (1946~),사람과 사람들,380391


In [4]:
remove_files = pd.read_excel('겹치는애들.xlsx')
table_data = table_data.loc[-table_data['이미지 고유 번호'].isin(remove_files['이미지 고유 번호']), :].reset_index(drop = True)
table_data.shape

(15197, 43)

### (2) 이미지 데이터

In [4]:
folder_list = sorted(os.listdir('data/image'))[3:] + sorted(os.listdir('data/image'))[:3]
folder_list

['1월_files',
 '2월_files',
 '3월_files',
 '4월_files',
 '5월_files',
 '6월_files',
 '7월_files',
 '8월_files',
 '9월_files',
 '10월_files',
 '11월_files',
 '12월_files']

In [5]:
serial = []
image_dir = []

for folder_name in folder_list:
    file_list = os.listdir(os.path.join('data/image', folder_name))
    for fname in file_list:
        if os.path.getsize(os.path.join(os.path.join('data/image', folder_name), fname)) > 3200:
            serial.append(fname[:-4])
            image_dir.append(os.path.join(os.path.join('data/image', folder_name), fname))

image_df = pd.DataFrame({'이미지 고유 번호': serial, '이미지경로': image_dir})
image_df.head()

,이미지 고유 번호,이미지경로
0,335298,data/image/1월_files/335298.jpg
1,340721,data/image/1월_files/340721.jpg
2,357485,data/image/1월_files/357485.jpg
3,357663,data/image/1월_files/357663.jpg
4,155,data/image/1월_files/155.jpg


### (3) 합치기

In [6]:
table_data['이미지 고유 번호'] = table_data['이미지 고유 번호'].map(str)
final_data = pd.merge(table_data, image_df)
final_data.head()

,작가생존여부_사망,작가생존여부_생존,작가생존여부_알수없음,판매계절_가을,판매계절_겨울,판매계절_봄,판매계절_여름,재료_견본채색,재료_기타,재료_브론즈,...,판매처_케이옥션,판매처_헤럴드아트데이,가로,세로,작품 판매 횟수,판매가격,작가명,제목,이미지 고유 번호,이미지경로
0,1,0,0,1,0,0,0,0,0,0,...,0,0,0.065372,0.065372,0.0,600000,임상진 Lim SangChin (1935~2013),무제,380410,data/image/11월_files/380410.jpg
1,1,0,0,1,0,0,0,0,0,0,...,0,0,0.026429,0.026429,0.0,400000,정술원 Jung SulWon (1885~1959),화조,380460,data/image/11월_files/380460.jpg
2,1,0,0,1,0,0,0,0,0,0,...,0,0,0.018878,0.018878,0.0,100000,정주상 Jeong JuSang (1925~2012),심정흥장 (선면),380491,data/image/11월_files/380491.jpg
3,0,1,0,1,0,0,0,0,0,0,...,0,0,0.025351,0.025351,0.0,360000,이양원 Lee YangWon (1944~),풍속도,380417,data/image/11월_files/380417.jpg
4,0,1,0,1,0,0,0,0,1,0,...,0,0,0.019957,0.019957,0.0,240000,이외수 Lee OiSoo (1946~),사람과 사람들,380391,data/image/11월_files/380391.jpg


### (4) 최종 데이터 저장

In [41]:
final_data.to_excel('final_data.xlsx', index = False, encoding = 'euc-kr')

## 3. Image 모델링

### (1) Dataset 생성

In [5]:
final_data = pd.read_excel('final_data_rgb_hsv.xlsx')
image_dir = final_data['이미지경로']
target = np.log10(final_data['판매가격'])

In [6]:
train_image_dir, test_image_dir, train_target, test_target = train_test_split(image_dir, target, train_size = 0.8, random_state = 1004)

In [7]:
train_image_dataset_generator = DatasetGenerator(list(train_image_dir), list(train_target), batch_size = 16, phase = 'train', train_valid_split = False)
train_dataloader = train_image_dataset_generator.dataloader()

test_image_dataset_generator = DatasetGenerator(list(test_image_dir), list(test_target), batch_size = 1, phase = 'test', train_valid_split = False)
test_dataloader = test_image_dataset_generator.dataloader()

### (2) Model 생성

In [9]:
model_name = 'resnet18'

In [8]:
# build model
vision_model = models.resnet18(pretrained=True)
num_ftrs = vision_model.fc.in_features
vision_model.fc = nn.Linear(num_ftrs, 1)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /home/jongwook95.lee/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:02<00:00, 22.7MB/s]


### (3) 학습 파라미터 지정

In [10]:
epoch = 10
learning_rate = 0.001
weight_decay = 0.0001
result_dir = './result/'

### (4) Loss, Optimizer 생성

In [11]:
# get loss function from LossFactory
loss_fn = L1Loss()

# get optimizer from OptimizerFactory
optimizer = Adam(params = vision_model.parameters(),
                lr=learning_rate,
                weight_decay = weight_decay)

### (5) 학습

In [12]:
print("{} start training!".format(model_name))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vision_model.to(device)
min_valid_loss = np.inf

# training
for e in range(epoch):
    train_loss = 0.0
    vision_model.train()     # Optional when not using Model Specific layer
    for data in tqdm(train_dataloader['train']):
        if torch.cuda.is_available():
            images, labels = data['image'].float().to(device), data['target'].float().to(device)
        
        optimizer.zero_grad()
        target = vision_model(images)
        loss = loss_fn(target,labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() / len(images)
            
    valid_loss = 0.0
    vision_model.eval()     # Optional when not using Model Specific layer
    for data in tqdm(test_dataloader['test']):
        if torch.cuda.is_available():
            data, labels = data['image'].float().to(device), data['target'].float().to(device)
        
        target = vision_model(data)
        loss = loss_fn(target,labels)
        valid_loss = loss.item() * len(data)

    print("Epoch: {}, Training Loss: {}, Test Loss: {}".format(e+1, train_loss / len(train_dataloader['train']), valid_loss))   
    if min_valid_loss > valid_loss:
        print(f'Validation Loss Decreased({min_valid_loss:.6f}--->{valid_loss:.6f}) \t Saving The Model')
        min_valid_loss = valid_loss
        # Saving State Dict
        torch.save(vision_model.state_dict(), result_dir + 'Best_image_resnet18_model.pth')    

resnet18 start training!


  0%|          | 0/752 [00:00<?, ?it/s]/home/jongwook95.lee/miniforge3/envs/vision_machine/lib/python3.8/site-packages/torch/nn/modules/loss.py:96: UserWarning: Using a target size (torch.Size([16])) that is different to the input size (torch.Size([16, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
100%|█████████▉| 751/752 [01:45<00:00,  7.25it/s]/home/jongwook95.lee/miniforge3/envs/vision_machine/lib/python3.8/site-packages/torch/nn/modules/loss.py:96: UserWarning: Using a target size (torch.Size([10])) that is different to the input size (torch.Size([10, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
  0%|          | 0/3007 [00:00<?, ?it/s]/home/jongwook95.lee/miniforge3/envs/vision_machine/lib/python3.8/site-packages/torch/nn/modules/loss.py:9

Epoch: 1, Training Loss: 0.038340128287474844, Test Loss: 0.006724357604980469
Validation Loss Decreased(inf--->0.006724) 	 Saving The Model


100%|██████████| 3007/3007 [00:25<00:00, 116.06it/s]


Epoch: 2, Training Loss: 0.033259301622487365, Test Loss: 0.2789168357849121


...

Epoch: 8, Training Loss: 0.03277287238247772, Test Loss: 0.13595056533813477


100%|██████████| 3007/3007 [00:30<00:00, 99.50it/s] 


Epoch: 9, Training Loss: 0.032829817411232184, Test Loss: 0.0937657356262207


100%|██████████| 3007/3007 [00:27<00:00, 107.63it/s]

Epoch: 10, Training Loss: 0.03263158530090995, Test Loss: 0.21628379821777344


### (6) Load Best Model

In [13]:
vision_model.load_state_dict(torch.load('result/Best_image_resnet18_model.pth'))

<All keys matched successfully>

### (7) 성능 평가

#### - 학습 데이터에 대한 성능

In [14]:
print('start prediction')
predictions = []
vision_model.to(device)

with torch.no_grad():  
    for data in train_dataloader['train']:
        images, labels = data['image'].float().to(device), data['target'].float().to(device)
        images = images.to(device)  
        labels = labels.to(device)  
        vision_model.eval()  
        yhat = vision_model(images)  
        pred = list(yhat.cpu().numpy())
        predictions = predictions + list(np.hstack(pred))

start prediction


In [15]:
from sklearn.metrics import mean_squared_error, r2_score

print("RMSE: {}".format(np.sqrt(mean_squared_error(train_target, predictions))))
print("R2 Score: {}".format(r2_score(train_target, predictions)))

RMSE: 0.7030030122693514
R2 Score: -0.06844438364680894


#### - 테스트 데이터에 대한 성능

In [16]:
print('start prediction')
predictions = []
vision_model.to(device)

with torch.no_grad():  
    for data in test_dataloader['test']:
        images, labels = data['image'].float().to(device), data['target'].float().to(device)
        images = images.to(device)  
        labels = labels.to(device)  
        vision_model.eval()  
        yhat = vision_model(images)  
        pred = list(yhat.cpu().numpy())
        predictions.append(pred[0][0])

start prediction


In [17]:
from sklearn.metrics import mean_squared_error, r2_score

print("RMSE: {}".format(np.sqrt(mean_squared_error(test_target, predictions))))
print("R2 Score: {}".format(r2_score(test_target, predictions)))

RMSE: 0.7222993517405499
R2 Score: -0.06374131826472906


: 